In [1]:
import torch
import sys

print("Python version:", sys.version)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("GPU count:", torch.cuda.device_count())
    
    # Quick tensor test
    x = torch.randn(1000, 1000).cuda()
    y = torch.randn(1000, 1000).cuda()
    z = torch.matmul(x, y)
    print("GPU computation successful!")
else:
    print("No GPU detected!")

Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:40:08) [MSC v.1938 64 bit (AMD64)]
PyTorch version: 2.4.0+cpu
CUDA available: False
No GPU detected!


In [87]:
import os
import json
import pandas as pd
from datetime import datetime, timezone

def normalize_timestamp(timestamp):
    try:
        # Case 1: Check if it's a Unix timestamp in milliseconds or seconds
        if isinstance(timestamp, (int, float)) or (isinstance(timestamp, str) and timestamp.isdigit()):
            timestamp = int(timestamp)  # Ensure it's an integer
            if len(str(timestamp)) == 13:  # Milliseconds
                return datetime.fromtimestamp(timestamp / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            elif len(str(timestamp)) == 10:  # Seconds
                return datetime.fromtimestamp(timestamp, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

        # Case 2: Check if it's already in a recognizable datetime format
        elif isinstance(timestamp, str):
            # Try parsing with known formats
            for fmt in ['%Y-%m-%d %H:%M', '%Y-%m-%d %H:%M:%S']:
                try:
                    return datetime.strptime(timestamp, fmt).strftime('%Y-%m-%d %H:%M:%S')
                except ValueError:
                    continue

        # If none of the above cases match, raise an error
        raise ValueError(f"Unrecognized timestamp format: {timestamp}")

    except Exception as e:
        print(f"Error normalizing timestamp: {e}")
        return None


def normalize_data(df: pd.DataFrame) -> tuple[pd.DataFrame, bool]:
    """
    - standardize column names (lower/strip)
    - convert timestamp column to UTC datetime
    - sort rows by timestamp and reset the index
    - return whether a volume column is present
    """
    df = df.copy()
    df.columns = [col.strip().lower() for col in df.columns]

    timestamp_candidates = ["timestamp", "time", "date"]
    timestamp_col = next((c for c in timestamp_candidates if c in df.columns), None)
    if not timestamp_col:
        raise ValueError("No timestamp-like column found in dataframe.")

    ts_normalized = df[timestamp_col].apply(normalize_timestamp)
    df[timestamp_col] = pd.to_datetime(ts_normalized, utc=True, errors="coerce")
    df = df.dropna(subset=[timestamp_col]).sort_values(by=timestamp_col).reset_index(drop=True)

    if timestamp_col != "timestamp":
        df = df.rename(columns={timestamp_col: "timestamp"})

    if "volume" in df.columns:
        df = df.drop(columns=["volume"])

    return df


def parse_labels(json_file, folder_path):
    with open(os.path.join(folder_path, json_file), 'r', encoding='utf-8') as file:
        data = json.load(file)

    folder_dict = {}

    for item in data:
        # Determine format
        if 'annotations' in item:
            file_name = item['data']['csv']
            # Remove hash: split by '-' and take from second part
            file_name_clean = '-'.join(file_name.split('-')[1:]) if '-' in file_name else file_name
            for annotation in item['annotations']:
                for result in annotation['result']:
                    label_list = result['value']['timeserieslabels']
                    # Convert list to string (take first if single, or join)
                    label = label_list[0] if len(label_list) == 1 else ' '.join(label_list)
                    start_time = result['value']['start']
                    end_time = result['value']['end']
                    if file_name_clean not in folder_dict:
                        folder_dict[file_name_clean] = []
                    folder_dict[file_name_clean].append({
                        'label': label,
                        'start': pd.to_datetime(normalize_timestamp(start_time), utc=True),
                        'end': pd.to_datetime(normalize_timestamp(end_time), utc=True)
                    })
        elif 'label' in item:
            file_name = item['csv']
            # Remove hash: split by '-' and take from second part
            file_name_clean = '-'.join(file_name.split('-')[1:]) if '-' in file_name else file_name
            for label_entry in item['label']:
                label_list = label_entry['timeserieslabels']
                # Convert list to string (take first if single, or join)
                label = label_list[0] if len(label_list) == 1 else ' '.join(label_list)
                start_time = label_entry['start']
                end_time = label_entry['end']
                if file_name_clean not in folder_dict:
                    folder_dict[file_name_clean] = []
                folder_dict[file_name_clean].append({
                    'label': label,
                    'start': pd.to_datetime(normalize_timestamp(start_time), utc=True),
                    'end': pd.to_datetime(normalize_timestamp(end_time), utc=True)
                })
        else:
            print(f"Unknown format for item ID: {item.get('id', 'N/A')}")

    return folder_dict


def get_data_with_labels(data_path):
    folders = os.listdir(data_path)
    if 'consensus' in folders:
        folders.remove('consensus')
    if 'sample' in folders:
        folders.remove('sample')

    all_labels = {}
    for folder in folders:
        folder_path = os.path.join(data_path, folder)
        if os.path.isdir(folder_path):
            json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

            if not json_files:
                print(f'No JSON file found in folder: {folder}')
                continue
            print(f'Folder: {folder}, JSON files: {json_files}')

            folder_dict = {}
            for json_file in json_files:
                parsed_labels = parse_labels(json_file, folder_path)
                for file_name, labels in parsed_labels.items():
                    if file_name not in folder_dict:
                        folder_dict[file_name] = []
                    folder_dict[file_name].extend(labels)

            # Deduplicate labels within each file
            for file_name, labels in folder_dict.items():
                unique_labels = {frozenset(label.items()): label for label in labels}.values()
                folder_dict[file_name] = list(unique_labels)

            # Add the folder's data to the main dictionary
            all_labels[folder] = folder_dict

    data_with_labels = {}

    for folder, files in all_labels.items():
        folder_path = os.path.join(data_path, folder)
        for file_name, labels in files.items():
            file_path = os.path.join(folder_path, file_name)
            if os.path.exists(file_path):
                data = pd.read_csv(file_path)

                data = normalize_data(data)

                # Attach the labels to the data
                key = f"{folder}/{file_name}"
                data_with_labels[key] = {
                    'data': data,
                    'labels': labels
                }

    return data_with_labels

In [88]:
data_path = 'C:/msc_2/DL/vitmma19-pw-bullflag/data'
data_with_labels = get_data_with_labels(data_path)

Folder: AS76HW, JSON files: ['labels.json']
Folder: AY1PC8, JSON files: ['BullFlag_Detector_Labeling_AY1PC8.json']
Folder: DMQSIV, JSON files: ['eurusd_labeling.json', 'xau_labeling.json']
Folder: DO1H50, JSON files: ['DO1H50.json', 'labels.json']
Folder: GFTYRV, JSON files: ['project-4-at-2025-10-30-12-12-aaf7a367.json']
Folder: IJAKT3, JSON files: ['BullFlag_Detector_Labeling_IJAKT3.json']
No JSON file found in folder: ITWQ3V
Folder: IV92EM, JSON files: ['project-5-at-2025-10-15-22-34-c286e467.json', 'project-6-at-2025-10-30-13-01-6c010a39.json']
No JSON file found in folder: J2QIYD
Folder: JRG6JB, JSON files: ['labels.json']
Folder: LMIRPG, JSON files: ['LMIRPG.json']
No JSON file found in folder: processed
Folder: Q5C0UO, JSON files: ['labels.json']
Folder: TYEGJ8, JSON files: ['project-1-at-2025-10-15-20-45-1f5298af.json']
No JSON file found in folder: U4DUY6
Folder: UZ2DPI, JSON files: ['UZ2DPI.json']
Folder: VWXUD6, JSON files: ['min_label.json', 'project-2-at-2025-10-09-14-54-f

In [89]:
vol = 0
no_vol = 0
for file_name, content in data_with_labels.items():
    # print(f"File: {file_name}")
    # print("Data:")
    # print(content['data'].head(1))
    # print(content['data'].columns)
    # print("Labels:")
    # print(content['labels'])

    if "volume" in content['data'].columns:
        print(f"{file_name} contains volume data.")
        vol += 1
    else:
        print(f"{file_name} does not contain volume data.")
        no_vol += 1
print(f"Total with volume: {vol}, without volume: {no_vol}")

AS76HW/XAU_1h_data_limited.csv does not contain volume data.
AS76HW/XAU_1m_data_limited.csv does not contain volume data.
AS76HW/XAU_5m_data_limited.csv does not contain volume data.
AS76HW/XAU_15m_data_limited.csv does not contain volume data.
AS76HW/XAU_30m_data_limited.csv does not contain volume data.
AY1PC8/EURUSD_1H_005.csv does not contain volume data.
AY1PC8/EURUSD_1min_001.csv does not contain volume data.
DO1H50/EURUSD_1hour_001.csv does not contain volume data.
DO1H50/EURUSD_1minute_001.csv does not contain volume data.
GFTYRV/GSPC_2025-09-25_to_2025-09-29_5m.csv does not contain volume data.
GFTYRV/GSPC_2025-09-30_to_2025-10-04_5m.csv does not contain volume data.
GFTYRV/GSPC_2025-10-05_to_2025-10-09_5m.csv does not contain volume data.
GFTYRV/GSPC_2025-10-10_to_2025-10-14_5m.csv does not contain volume data.
GFTYRV/GSPC_2025-10-15_to_2025-10-19_5m.csv does not contain volume data.
GFTYRV/GSPC_2025-10-20_to_2025-10-24_5m.csv does not contain volume data.
GFTYRV/NDX_2025-09-

In [26]:
data_with_labels

{'AS76HW/XAU_1h_data_limited.csv': {'data':                        timestamp     open     high      low    close
  0      2004-06-11 07:00:00+00:00   384.00   384.30   383.30   383.80
  1      2004-06-11 08:00:00+00:00   383.80   384.30   383.10   383.10
  2      2004-06-11 09:00:00+00:00   383.10   384.10   382.80   383.10
  3      2004-06-11 10:00:00+00:00   383.00   383.80   383.00   383.60
  4      2004-06-11 11:00:00+00:00   383.60   383.80   383.50   383.60
  ...                          ...      ...      ...      ...      ...
  121818 2025-07-11 20:00:00+00:00  3353.68  3354.04  3349.22  3353.28
  121819 2025-07-11 21:00:00+00:00  3353.32  3358.51  3352.93  3356.13
  121820 2025-07-11 22:00:00+00:00  3356.15  3359.71  3355.62  3356.47
  121821 2025-07-11 23:00:00+00:00  3356.59  3357.97  3353.59  3357.59
  121822 2025-09-30 19:00:00+00:00  3843.36  3844.91  3842.41  3843.80
  
  [121823 rows x 5 columns],
  'labels': [{'label': 'Bearish Normal',
    'start': Timestamp('2006-06-0

Igazából ha minden címkéhez hozzá akarjuk adni a pole-t akkor a label előtti dolgokat kell nézni hogy ott van-e pole és azt hozzáadni. Ha pedig el akarjuk tüntetni a poleokat akkor a label elejét kell megnézni és ha ott van pole az első pár gyertyán akkor azt kiszedjük. Ezt akér lehet label arányosan csinálni hogy pl első 10%ot nézzük meg és abból azt a rész kiszedjük ami a legnagyobb meredekségű és abs álép egy határt.


In [ ]:
from typing import List, Dict
import numpy as np
import pandas as pd

def extend_labels_with_pole(
    ohlc_df: pd.DataFrame,
    labels: List[Dict],
    price_col: str = "close",
    timestamp_col: str = "timestamp",

    min_bar_slope: float = 0.001,      # ~0.1% move per bar
    min_total_gain: float = 0.01,      # ~1% pole gain
    max_pole_bars: int = 25
) -> List[Dict]:
    """
    For each label interval, scan backward to attach a pole if a steep move precedes the flag.
    Thresholds (`min_bar_slope`, `min_total_gain`) are derived dynamically from the label's own
    price action (percentiles of recent bar slopes and cumulative gains).
    """
    df = ohlc_df.copy()
    df = df.sort_values(timestamp_col).reset_index(drop=True)

    ts_values = df[timestamp_col].to_numpy()
    price = df[price_col].to_numpy()

    def timestamp_to_idx(ts):
        idx = np.searchsorted(ts_values, ts)
        if idx >= len(ts_values) or ts_values[idx] != ts:
            return None
        return idx

    updated_labels = []
    for entry in labels:
        start_idx = timestamp_to_idx(entry["start"])
        end_idx = timestamp_to_idx(entry["end"])
        if start_idx is None or end_idx is None:
            entry = {**entry, "pole_start": entry["start"], "pole_added": False, "pole_gain": 0.0}
            updated_labels.append(entry)
            continue

        direction = 1 if "bull" in entry["label"].lower() else -1
        pole_start_idx = start_idx
        steps = 0

        while pole_start_idx > 0 and steps < max_pole_bars:
            prev_idx = pole_start_idx - 1
            bar_return = direction * ((price[pole_start_idx] - price[prev_idx]) / price[prev_idx])
            if bar_return < min_bar_slope:
                break
            pole_start_idx = prev_idx
            steps += 1

        total_gain = direction * ((price[start_idx] - price[pole_start_idx]) / price[pole_start_idx])
        if total_gain >= min_total_gain and pole_start_idx != start_idx:
            pole_ts = ts_values[pole_start_idx].isoformat()
            updated_labels.append({
                **entry,
                "pole_start": pd.to_datetime(pole_ts, utc=True),
                "pole_added": True,
                "pole_gain": float(total_gain)
            })
        else:
            updated_labels.append({
                **entry,
                "pole_start": pd.to_datetime(entry["start"], utc=True),
                "pole_added": False,
                "pole_gain": float(total_gain)
            })

    return updated_labels


In [ ]:
def extend_labels_with_pole(
    ohlc_df: pd.DataFrame,
    labels: List[Dict],
    price_col: str = "close",
    timestamp_col: str = "timestamp",
    max_pole_bars: int = 25,
    label_window: int = 5,
    slope_multiplier: float = 1.25,
    gain_multiplier: float = 1.15,
    default_min_bar_slope: float = 0.0001,
    default_min_total_gain: float = 0.005,
) -> List[Dict]:
    df = ohlc_df.copy().sort_values(timestamp_col).reset_index(drop=True)
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], utc=True)

    ts_values = df[timestamp_col].to_numpy()
    price = df[price_col].to_numpy()

    def timestamp_to_idx(ts):
        idx = np.searchsorted(ts_values, ts)
        if idx >= len(ts_values) or ts_values[idx] != ts:
            return None
        return idx

    # ezt a logikát erőteljesen át kell gondolni mert lehet pont ellentétes irányú néha működik néha nem stb
    def label_threshold(entry):
        start_idx = timestamp_to_idx(entry["start"])
        end_idx = timestamp_to_idx(entry["end"])
        if start_idx is None or end_idx is None or end_idx <= start_idx:
            return default_min_bar_slope, default_min_total_gain

        direction = 1 if "bull" in entry["label"].lower() else -1
        last_idx = min(start_idx + label_window, end_idx)
        bar_slopes = []
        for i in range(start_idx + 1, last_idx + 1):
            bar_return = direction * ((price[i] - price[i - 1]) / price[i - 1])
            if bar_return > 0:
                bar_slopes.append(bar_return)

        base_slope = np.median(bar_slopes) if bar_slopes else default_min_bar_slope
        label_gain = direction * ((price[last_idx] - price[start_idx]) / price[start_idx])
        base_gain = label_gain if label_gain > 0 else default_min_total_gain

        return base_slope * slope_multiplier, base_gain * gain_multiplier

    updated_labels = []
    for entry in labels:
        min_bar_slope, min_total_gain = label_threshold(entry)

        start_idx = timestamp_to_idx(entry["start"])
        end_idx = timestamp_to_idx(entry["end"])
        if start_idx is None or end_idx is None:
            updated_labels.append({**entry, "pole_start": entry["start"], "pole_added": False, "pole_gain": 0.0})
            continue

        direction = 1 if "bull" in entry["label"].lower() else -1
        pole_start_idx = start_idx
        steps = 0

        while pole_start_idx > 0 and steps < max_pole_bars:
            prev_idx = pole_start_idx - 1
            bar_return = direction * ((price[pole_start_idx] - price[prev_idx]) / price[prev_idx])
            if bar_return < min_bar_slope:
                break
            pole_start_idx = prev_idx
            steps += 1

        total_gain = direction * ((price[start_idx] - price[pole_start_idx]) / price[pole_start_idx])
        if total_gain >= min_total_gain and pole_start_idx != start_idx:
            updated_labels.append({
                **entry,
                "pole_start": ts_values[pole_start_idx],
                "pole_added": True,
                "pole_gain": float(total_gain),
            })
        else:
            print(f"Bar Return: {bar_return} Min Bar Slope: {min_bar_slope}, Total gain: {total_gain} Min Total Gain: {min_total_gain}")
            updated_labels.append({
                **entry,
                "pole_start": entry["start"],
                "pole_added": False,
                "pole_gain": float(total_gain),
            })

    return updated_labels

In [ ]:
data = data_with_labels["VWXUD6/XAUUSD_5m_001.csv"]['data']
labels = data_with_labels["VWXUD6/XAUUSD_5m_001.csv"]['labels']

extended_labels = extend_labels_with_pole(data, labels)
extended_labels

In [ ]:
# Milyen modellel dolgozunk? és hogyan várja az adatokat?
# Nem mindehol van volume oszlop mert ez is elég szarul volt specifikálva a feladatkiírásban. Tehát lehet még azt is ki kell dobni ahol van
# maradjunk a data with labelsnél majd kicsit átírjuk az adatfeldolgozást ha jól működik ez a pole hozzáadás és kell is


In [90]:
from typing import List, Dict

def attach_interval_labels(
    ohlc_df: pd.DataFrame,
    labels: List[Dict],
    label_map: Dict[str, int],
    timestamp_col: str = "timestamp",
    target_col: str = "flag_label"
) -> pd.DataFrame:
    """
    Sor-szintű címkézés: minden sornak megadja a hozzá tartozó flag ID-t.
    Feltételezi, hogy a DataFrame-ben van timestamp oszlop.
    """
    df = ohlc_df.copy()
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], utc=True)
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    df[target_col] = label_map.get("No Flag", 0)

    for entry in labels:
        start = pd.to_datetime(entry["start"], utc=True)
        end = pd.to_datetime(entry["end"], utc=True)
        cls_id = label_map.get(entry["label"], label_map.get("No Flag", 0))
        mask = (df[timestamp_col] >= start) & (df[timestamp_col] <= end)
        df.loc[mask, target_col] = cls_id

    return df

label_map = {
    "No Flag": 0,
    "Bullish Normal": 1,
    "Bullish Wedge": 2,
    "Bullish Pennant": 3,
    "Bearish Normal": 4,
    "Bearish Wedge": 5,
    "Bearish Pennant": 6,
}


In [60]:
labeled_df

,timestamp,open,high,low,close,volume,flag_label
0,2023-02-27 01:00:00+00:00,1812.62,1813.18,1811.86,1812.06,79,0
1,2023-02-27 01:05:00+00:00,1811.65,1812.16,1811.46,1811.49,106,0
2,2023-02-27 01:10:00+00:00,1811.49,1812.83,1811.16,1812.73,268,0
3,2023-02-27 01:15:00+00:00,1812.76,1813.10,1812.17,1812.41,183,0
4,2023-02-27 01:20:00+00:00,1812.33,1812.33,1811.75,1812.04,132,0
...,...,...,...,...,...,...,...
13517,2023-05-05 23:35:00+00:00,2016.42,2016.56,2015.89,2016.41,148,0
13518,2023-05-05 23:40:00+00:00,2016.41,2016.64,2016.13,2016.45,125,0
13519,2023-05-05 23:45:00+00:00,2016.40,2016.49,2015.78,2016.47,175,0
13520,2023-05-05 23:50:00+00:00,2016.56,2016.70,2015.62,2015.83,152,0


In [102]:
from sklearn.preprocessing import StandardScaler


def split_time_series(
    df: pd.DataFrame,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    timestamp_col: str = "timestamp"
) -> dict[str, pd.DataFrame]:
    df_sorted = df.sort_values(timestamp_col).reset_index(drop=True)
    n = len(df_sorted)
    train_end = int(n * train_ratio)
    val_end = train_end + int(n * val_ratio)
    return {
        "train": df_sorted.iloc[:train_end].copy(),
        "val": df_sorted.iloc[train_end:val_end].copy(),
        "test": df_sorted.iloc[val_end:].copy(),
    }


def generate_windows(
    df: pd.DataFrame,
    feature_cols: list[str],
    target_col: str,
    window_size: int = 64,
    step: int = 1
) -> tuple[np.ndarray, np.ndarray]:
    features = df[feature_cols].to_numpy(dtype=np.float32)
    targets = df[target_col].to_numpy()
    X, y = [], []
    for end_idx in range(window_size, len(df), step):
        start_idx = end_idx - window_size
        window = features[start_idx:end_idx]
        target = targets[end_idx - 1]
        X.append(window)
        y.append(target)
    return np.stack(X), np.array(y)


def normalize_splits(
    splits: dict[str, pd.DataFrame],
    feature_cols: list[str]
) -> tuple[dict[str, pd.DataFrame], StandardScaler]:
    scaler = StandardScaler()
    train_features = splits["train"][feature_cols]
    scaler.fit(train_features)
    normalized = {}
    for key, split_df in splits.items():
        norm_df = split_df.copy()
        norm_df[feature_cols] = scaler.transform(split_df[feature_cols])
        normalized[key] = norm_df
    return normalized, scaler


In [ ]:
from datetime import datetime, timezone
from pathlib import Path
import joblib

window_size = 32
step = 1
data_path = Path("C:/msc_2/DL/vitmma19-pw-bullflag/data")
output_dir = Path(f"{data_path}/processed")
output_dir.mkdir(parents=True, exist_ok=True)

train_ratio = 0.7
val_ratio = 0.15

features = ['open', 'high', 'low', 'close']

window_arrays = {
    "train_X": [],
    "train_y": [],
    "val_X": [],
    "val_y": [],
    "test_X": [],
    "test_y": [],
}

for file_name, content in data_with_labels.items():
    data = content['data']
    labels = content['labels']

    # ez majd ha jól működik akkor kell
    # extended_labels = extend_labels_with_pole(data, labels)
    # print(f"File: {file_name}, Original labels: {len(labels)}, Extended labels: {len(extended_labels)}")

    labeled_df = attach_interval_labels(data, labels, label_map)

    if len(labeled_df["flag_label"].unique()) < 2 and labeled_df["flag_label"].unique()[0] == 0:
        print(f"Skip {file_name}: only 0 class present.")
        continue

    df_len = len(labeled_df)

    if df_len < window_size:
        print(f"Skip {file_name}: insufficient rows ({df_len}) for window_size={window_size}")
        continue

    if df_len * min(train_ratio, val_ratio, (1 - train_ratio - val_ratio)) < window_size:
        # skip file splitting use all for training
        splits = {"train": labeled_df}
    else:
        splits = split_time_series(labeled_df, train_ratio=train_ratio, val_ratio=val_ratio)

    normalized_splits, scaler = normalize_splits(splits, feature_cols=features)

    for split_name, split_df in normalized_splits.items():
        if len(split_df) < window_size:
            print(f"Skip {split_name}: insufficient rows ({len(split_df)}) for window_size={window_size}")
            continue
        X, y = generate_windows(split_df, feature_cols=features, target_col="flag_label", window_size=window_size, step=step)
        
        window_arrays[f"{split_name}_X"].append(X)
        window_arrays[f"{split_name}_y"].append(y)


In [ ]:
len(window_arrays["test_X"])

18242

In [ ]:
stacked_train_X = np.concatenate(window_arrays["train_X"], axis=0)
stacked_train_y = np.concatenate(window_arrays["train_y"], axis=0)
print(stacked_train_X.shape, stacked_train_y.shape)

stacked_val_X = np.concatenate(window_arrays["val_X"], axis=0)
stacked_val_y = np.concatenate(window_arrays["val_y"], axis=0)
print(stacked_val_X.shape, stacked_val_y.shape)

stacked_test_X = np.concatenate(window_arrays["test_X"], axis=0)
stacked_test_y = np.concatenate(window_arrays["test_y"], axis=0)
print(stacked_test_X.shape, stacked_test_y.shape)


(3285263, 32, 4) (3285263,)
(702434, 32, 4) (702434,)
(702513, 32, 4) (702513,)


In [128]:
np.savez_compressed(
    data_path / "processed_data.npz",
    train_X=stacked_train_X,
    train_y=stacked_train_y,
    val_X=stacked_val_X,
    val_y=stacked_val_y,
    test_X=stacked_test_X,
    test_y=stacked_test_y,
)

In [134]:
DATA_DIR = Path("C:/msc_2/DL/vitmma19-pw-bullflag/data")
PROCESSED_FILE = DATA_DIR / "processed_data.npz"


def load_processed_dataset(npz_file: Path) -> dict[str, np.ndarray]:
    if not npz_file.exists():
        raise FileNotFoundError(f"Missing file: {npz_file}")
    with np.load(npz_file) as data:
        return {key: data[key] for key in data.files}


dataset = load_processed_dataset(PROCESSED_FILE)
for key, array in dataset.items():
    print(f"{key}: shape={array.shape}, dtype={array.dtype}")

train_X: shape=(3285263, 32, 4), dtype=float32
train_y: shape=(3285263,), dtype=int64
val_X: shape=(702434, 32, 4), dtype=float32
val_y: shape=(702434,), dtype=int64
test_X: shape=(702513, 32, 4), dtype=float32
test_y: shape=(702513,), dtype=int64
